In [4]:
import numpy as np
import pandas as pd
from GeoSolver import GeoSolver

In [5]:
class Object():
    """Описывает один материал в заказе"""
    
    def __init__(self, material, count):
        """
        материал - колличество
        """
        self.material = material
        self.count = count

In [6]:
class Lot():
    
    """Описывает один лот"""
    
    def __init__(self, material, count, date, coords, providers):
        """
        материал и его кол-во, срок поставки, клиента (грузополучателя)
        Создает объет матерала
        Формирует словарь возможных поставщиков и кол-во товаров, которые они могут 
        закупить из переданной таблцы материал-поставщик (мб None!)
        Инициализирует географический центр
        Инициализируется минимальная и максимальная дата доставки
        """
        
        self.__content = [Object(material, count)]
        self.__min_date, self.__max_date = date, date
        self.__center = coords
        if len(coords) == 0:
            print(f'{material} {coords}!!!!!!!')
            self.__center = np.array([0, 0])
        self.__providers = {p : 1 for p in providers}
        self.__order_number = 1 # Количество позиций     
    
    def merge(self, other):
        """
        Сливает other в теекущий лот
        Обновляет центр, минимальные даты
        Сливает словари
        Добавляет список товаров
        """
        self.__providers = self.providers_merge(other.get_providers())
        min_date_other, max_date_other = other.get_dates()
        self.__min_date, self.__max_date = min(self.__min_date, min_date_other), max(self.__max_date, max_date_other)
        self.__order_number += other.get_order_number()
        self.__content += other.get_content()
        self.__center = (self.__center + other.get_center()) / 2

    def providers_merge(self, other_providers):
        """
        получает словарь поставщиков и сливает его со своим
        """
        for key in self.__providers.keys():
            if key in other_providers.keys():
                other_providers[key] += self.__providers[key]
            else:
                other_providers[key] = self.__providers[key]
        return other_providers
    
    def get_center(self):
        return self.__center
    
    def get_dates(self):
        return self.__min_date, self.__max_date
        
    def get_providers(self):
        return self.__providers
        
    def get_order_number(self):
        return self.__order_number
        
    def get_content(self):
        return self.__content

In [16]:
class Solver():
    """
    Модель 1. Агломеративная кластеризация
    """
    
    def __init__(self, prod_percent = 50, prov_percent = 50, podgon = 1e-4):
        """
        Подгружает таблицу грузополучатель/координаты
        Подгружает справочник материал/поставщик
        Инифицализирует пустым списком список лотов
        """
        self.__coords = pd.read_csv("./Data/coords_test.csv")
        self.__providers = pd.read_csv("./Data/Кабель-справочник-МТР-refactored.csv")
        self.__lots = []
        self.__geosolver = GeoSolver()
        self.__prod_percent = prod_percent / 100
        self.__prov_percent = prov_percent / 100
        self.__podgon = podgon
    
    def get_lots(self, filename):
        """
        Получает имя файла в формате csv
        Запускает __file_handler()
        Запускает __construct_lots()
        возвращает то, что лежит в лотах
        """
        
        self.__file_handler(filename)
        self.__construct_lots()
        
        return self.__lots
        
    def get_distance(self, lot1, lot2):
        """
        Если None, то None
        Принимает на вход еще один лот, возвращает расстояние между ними
        Смотрит на разницу минимальной и максимальной даты доставки  <= 30
        Проверяется Условие Качества (50% выкупают 50% если объединить)
        Считается расстояние
        Возвращается расстояние 
        """
        if lot2 is None or (max(lot1.get_dates()[1], lot2.get_dates()[1]) - min(lot1.get_dates()[0], lot2.get_dates()[0])).days > 30:
            return None
        if (np.array(list(lot1.providers_merge(lot2.get_providers()).values())) >
            ((lot1.get_order_number() + lot2.get_order_number()) * self.__prod_percent)).mean() < self.__prov_percent: 
            # Меньше prov_percent выкупают prod_percent
            return None
        #print(lot1.get_center(), lot2.get_center())
        distance_real = self.__geosolver.find_distance(lot1.get_center(), lot2.get_center())
        prov_intersection = set(lot1.get_providers().keys()) & set(lot2.get_providers().keys())
        if len(prov_intersection) == 0:
            return None
        return (self.__podgon * distance_real ** 2 + 
                (len(set(lot1.get_providers().keys()) | set(lot2.get_providers().keys())) / len(prov_intersection) - 1) ** 2) ** 0.5
    
    def __file_handler(self, filename):
        """
        Открывает csv файл, делает датафрейм
        Проходит последовательно по строкам датафрейма, создавая лоты
        Лоты создаются в список self.__lots
        """
        
        data = pd.read_csv(filename)
        data['Срок поставки'] = pd.to_datetime(data['Срок поставки'])
        for i in data.index:
            material, count, client, date = data['Материал'][i], data['Общее количество'][i], \
                                            data['Грузополучатель'][i], data['Срок поставки'][i]
            #print(client)
            #print(self.__coords[self.__coords['Код грузополучателя'] == client]['Широта'])
            line = self.__coords[self.__coords['Код грузополучателя'] == client]
            coords = np.concatenate([line['Широта'].values, line['Долгота'].values])
            providers = self.__providers[self.__providers['Материал'] == material]['Поставщики']
            self.__lots.append(Lot(material, count, date, coords, providers))
    
    def __construct_lots(self):
        """
        Идет цикл с проверкой удовлетворения условия
        {
        вызввается __calc_min_distance(self)
        если None то мы закончили
        иначе lot[i].merge(lot[j])
        }
        """
        
        best_option = self.__calc_min_distance()
        while not best_option is None:
            i, j = best_option[0], best_option[1]
            print(f'{i} -- {j}')
            self.__lots[i].merge(self.__lots[j])
            self.__lots[j] = None
            best_option = self.__calc_min_distance()
            

    def __calc_min_distance(self):
        """
        Создает булевый список посещенных лотов
        Проходится по списку лотов
        Между каждой парой вызывает lot.get_distance(other)
        Запоминает и возвращает оптимальную пару или None
        """
        min_dist, opt1, opt2 = 1e10, -1, -1
        for i, first in enumerate(self.__lots):
            if first is None:
                continue
            for j, second in enumerate(self.__lots[i+1:]):
                dist = self.get_distance(first, second)
                if not dist is None and dist < min_dist:
                    min_dist = dist
                    opt1, opt2 = i, j + i + 1                    
        if opt1 == -1:
            return None
        return (opt1, opt2)

# Код

In [17]:
solver = Solver()

In [18]:
lots = solver.get_lots('./Data/2020-2.csv')

770000464608 []!!!!!!!
0 -- 2
0 -- 1
4 -- 5
12 -- 13
15 -- 16
17 -- 18
17 -- 19
17 -- 20
23 -- 24
25 -- 26
25 -- 27
25 -- 28
25 -- 29
25 -- 30
25 -- 31
37 -- 38
39 -- 40
39 -- 41
42 -- 43
42 -- 44
42 -- 45
49 -- 50
49 -- 51
49 -- 52
49 -- 53
49 -- 54
49 -- 55
60 -- 61
65 -- 66
67 -- 68
71 -- 72
71 -- 73
71 -- 74
75 -- 76
75 -- 77
75 -- 78
75 -- 79
75 -- 80
75 -- 81
75 -- 82
75 -- 83
75 -- 84
87 -- 88
90 -- 91
97 -- 98
97 -- 99
100 -- 101
102 -- 103
105 -- 106
109 -- 110
114 -- 115
120 -- 121
124 -- 137
124 -- 138
128 -- 129
128 -- 130
128 -- 131
128 -- 132
128 -- 133
128 -- 134
128 -- 135
128 -- 136
141 -- 142
141 -- 143
141 -- 144
145 -- 146
145 -- 147
150 -- 151
161 -- 162
161 -- 163
164 -- 165
164 -- 166
164 -- 167
168 -- 169
168 -- 170
168 -- 171
168 -- 172
173 -- 174
173 -- 175
173 -- 176
173 -- 177
173 -- 178
173 -- 179
173 -- 180
173 -- 181
173 -- 182
173 -- 183
173 -- 184
173 -- 185
173 -- 186
173 -- 187
173 -- 188
173 -- 189
173 -- 190
173 -- 191
173 -- 192
173 -- 193
173 -- 1

# Релуьтат

In [24]:
for i in lots:
    if not i is None:
        print(f'------------------------')
        for j in i.get_content(): 
            print(j.material, j.count)

------------------------
770000794338 0.288
770000607731 0.041
770000874938 0.338
770000697184 0.041
770000607790 0.455
770000607773 0.385
770000853293 0.21
770000261425 100.0
770000868418 2.0
------------------------
770000872682 0.5
770000872789 3.2
770000479467 2.0
770000027333 10.0
770000294792 0.5
770000568321 2.3
770000683433 4.0
770000890343 10.0
770000890313 4.0
770000299132 0.4
770000027333 1.0
770000122449 0.5
770000126918 0.1
770000872816 0.5
770000255507 0.5
770000583018 0.4
770000641981 1.0
770000122449 0.5
770000126918 0.1
770000583018 0.4
770000641981 1.0
770000470937 0.5
770000122449 0.5
770000126918 0.1
770000583018 0.4
770000641981 1.0
770000470937 0.5
770000122449 0.5
770000126918 0.1
770000583018 0.4
770000641981 1.0
770000470937 0.5
770000122449 0.5
770000126918 0.1
770000255507 0.5
770000255507 0.5
770000255507 0.5
770000255507 0.5
770000583018 0.4
770000641981 1.0
770000470937 0.5
770000122449 0.5
770000126918 0.1
770000583018 0.4
770000641981 1.0
770000470937 0.